# Projet

### Imports

In [57]:
# Imports
import glob, os
import pandas
import urllib
from SPARQLWrapper import SPARQLWrapper
import PIL.Image

## 1. Collecte de données

### Dataframe Functions

In [58]:
def jsonToDataframe(path):   
    with open(path, "r", encoding="utf-8") as f:
        dataframe = pandas.read_json(f, orient='index')
    return dataframe

def dataframeToJson(path, dataframe):   
    with open(path, "w", encoding="utf-8") as f:
        f.write(dataframe.to_json(orient='index', indent=2))

### Récupération du résultat d'une query Wikidata

In [59]:
size = 5
dataPath = "data.json"
imagesPath = "./images/"

# Query de Wikidata
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
query = """
SELECT ?item ?itemLabel ?pic WHERE {
  ?item wdt:P31 wd:Q4989906;
    wdt:P18 ?pic.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT """
# On précise le nombre d'entrées que l'on veut
query = query + str(size)

# On récupère le résultat de la query
sparql.setQuery(query)
sparql.setReturnFormat("json")
results = sparql.query().convert()

# On normalise les résultat pour pandas
array = []
for data in results["results"]["bindings"]:
    name = data["itemLabel"]["value"]
    format = data["pic"]["value"].split(".")[-1]
    path = f"{imagesPath}{name}.{format}"

    array.append([name,  
                  data["pic"]["value"], 
                  data["item"]["value"].split(sep='/')[-1],
                  format,
                  path,
                  None,
                  ['#monument']])
                  

dataframe = pandas.DataFrame(array, columns=["label", "link", "entityID", "format", "path", "size", "tags"])

# On enregistre les données au format JSON
dataframeToJson(dataPath, dataframe)

# Nettoyage du dossier /images
if not os.path.isdir('./images'):
  os.mkdir('./images')

# Nettoyage du dossier /images
else:
  files = glob.glob('./images/*')
  for f in files:
      os.remove(f)



dataframe


,label,link,entityID,format,path,size,tags
0,"Castle of Carlos V, Hondarribia",http://commons.wikimedia.org/wiki/Special:File...,Q1049154,jpg,"./images/Castle of Carlos V, Hondarribia.jpg",None,[#monument]
1,Castle of Chinchilla,http://commons.wikimedia.org/wiki/Special:File...,Q1049162,JPG,./images/Castle of Chinchilla.JPG,None,[#monument]
2,Castell de Forna,http://commons.wikimedia.org/wiki/Special:File...,Q1049184,jpg,./images/Castell de Forna.jpg,None,[#monument]
3,Castle of Jimena de la Frontera,http://commons.wikimedia.org/wiki/Special:File...,Q1049191,JPG,./images/Castle of Jimena de la Frontera.JPG,None,[#monument]
4,Castle of Gibralpharo,http://commons.wikimedia.org/wiki/Special:File...,Q1049197,jpg,./images/Castle of Gibralpharo.jpg,None,[#monument]


### Téléchargement des Images

In [62]:
dataframe = jsonToDataframe(dataPath)
for i in range(len(dataframe.index)):
    # On télécharge l'image
    link = dataframe._get_value(i,"link")
    path = dataframe._get_value(i,"path")
    response = urllib.request.urlretrieve(link, path)

,label,link,entityID,format,path,size,tags
0,"Castle of Carlos V, Hondarribia",http://commons.wikimedia.org/wiki/Special:File...,Q1049154,jpg,"./images/Castle of Carlos V, Hondarribia.jpg",NaN,[#monument]
1,Castle of Chinchilla,http://commons.wikimedia.org/wiki/Special:File...,Q1049162,JPG,./images/Castle of Chinchilla.JPG,NaN,[#monument]
2,Castell de Forna,http://commons.wikimedia.org/wiki/Special:File...,Q1049184,jpg,./images/Castell de Forna.jpg,NaN,[#monument]
3,Castle of Jimena de la Frontera,http://commons.wikimedia.org/wiki/Special:File...,Q1049191,JPG,./images/Castle of Jimena de la Frontera.JPG,NaN,[#monument]
4,Castle of Gibralpharo,http://commons.wikimedia.org/wiki/Special:File...,Q1049197,jpg,./images/Castle of Gibralpharo.jpg,NaN,[#monument]


### Récupération des Données EXIFS

In [61]:
dataframe = jsonToDataframe(dataPath)

wantedExifs = [274, 315, 306, 272]
columnNames = ["orientation", "artist", "date", 'appareil']

for i,tag in enumerate(wantedExifs):
    dataframe[columnNames[i]] = None

# On parcours les entrées
for i in range(len(dataframe.index)):
    path = dataframe._get_value(i,"path")
    img = PIL.Image.open(path)
    exif_data = img._getexif()
    img.close()

    dataframe.at[i, "size"] = img.size
    if exif_data:
      for tag in exif_data:
          if tag in wantedExifs:
              dataframe.at[i, columnNames[wantedExifs.index(tag)]] = exif_data[tag]

# On enregistre les données au format JSON
dataframeToJson(dataPath)
        
dataframe 

ValueError: Must have equal len keys and value when setting with an iterable

## Analyse des Données

In [ ]:
# Création de la dataframe des utilisateurs
usersPath = 'users.json'

array = [['default',[]]]
usersDataframe = pandas.DataFrame(array, columns=['username', 'likedImages'])

dataframeToJson(usersPath, usersDataframe)

usersDataframe

,username,likedImages
0,default,[]
